# Install libs

We need some libs to expose a Flask URL outside of google colab.  
Ngrok will provide a URL (free ngrok account needed).  
Flask-CORS will allow requests coming from a cross origin.  
pyngrok allows the storing of user-credentials.

In [7]:
!pip install Flask flask_ngrok
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!pip install Flask-CORS
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
!ngrok authtoken "INSERT_PERSONAL_TOKEN_HERE"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Launch server

Logs are displayed here so you can check whether the route sends back 200 or 500 status codes, or if you are calling a bad route from your front end.  

In [18]:
import base64
import io
import numpy as np
from PIL import Image
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from flask_ngrok import run_with_ngrok
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)   

# Load the saved Keras model
model = load_model('mnist_model.h5')

def preprocess_image(image):
    image = image.resize((28, 28), Image.ANTIALIAS).convert('L')
    image = np.array(image)
    image = image.reshape(1, 28, 28, 1)
    image = image.astype('float32')
    image /= 255
    return image

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    image_file = request.files['image']
    image = Image.open(io.BytesIO(image_file.read()))

    preprocessed_image = preprocess_image(image)
    prediction = model.predict(preprocessed_image)
    predicted_class = np.argmax(prediction, axis=-1)[0]

    return jsonify({'prediction': int(predicted_class)})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3a1f-35-196-229-219.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [15/Mar/2023 03:18:25] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Mar/2023 03:18:25] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 99ms/step


...

INFO:werkzeug:127.0.0.1 - - [15/Mar/2023 03:20:34] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 27ms/step


INFO:werkzeug:127.0.0.1 - - [15/Mar/2023 03:24:16] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Mar/2023 03:26:09] "GET / HTTP/1.1" 404 -


1/1 [==============================] - 0s 26ms/step


INFO:werkzeug:127.0.0.1 - - [15/Mar/2023 03:26:36] "POST /predict HTTP/1.1" 200 -


# Here is the HTML file that is needed

This page allows the upload of an image and displays the current digit inference.  

Copy-paste the current API address, something like `http://3a1f-35-196-229-219.ngrok.io/` in the provided form on the page.  

On image upload, a result will be displayed.  

````
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>MNIST Prediction</title>
</head>
<body>
  <h1>MNIST Prediction</h1>
  <form id="server-url-form">
    <label for="server-url-input">API Server URL:</label>
    <input type="text" id="server-url-input" value="http://127.0.0.1:5000" required>
  </form>
  <form id="image-upload-form">
    <label for="image-input">Upload an image:</label>
    <input type="file" id="image-input" accept="image/*">
  </form>
  <div id="result"></div>
  <script>
    async function sendImage(imageFile, serverUrl) {
      const formData = new FormData();
      formData.append('image', imageFile);

      const response = await fetch(`${serverUrl}/predict`, {
        method: 'POST',
        body: formData
      });

      if (!response.ok) {
        throw new Error(`HTTP error: ${response.status}`);
      }

      const result = await response.json();
      return result.prediction;
    }

    document.getElementById('image-input').addEventListener('change', async (event) => {
      const imageFile = event.target.files[0];
      const serverUrl = document.getElementById('server-url-input').value;

      if (imageFile) {
        try {
          const prediction = await sendImage(imageFile, serverUrl);
          document.getElementById('result').textContent = `Prediction: ${prediction}`;
        } catch (error) {
          console.error('Error sending image:', error);
          document.getElementById('result').textContent = 'Error: Failed to get a prediction.';
        }
      } else {
        document.getElementById('result').textContent = '';
      }
    });
  </script>
</body>
</html>
````